# Figure 3

**Figure 3 | Spearman correlation strength of sEEG to simulated LFP.** (A) Bottom-up sensory input model similarity to neural signals. sEEG electrode color intensity represents the similarity of the neural signal to the LFP simulated from the bottom-up model. Similarity is quantified as the absolute value of the median Spearman coefficient (ρ) across all included trials. Similarity is z-scored across n = 7 participants. Electrode size represents the FDR corrected p-value from a 1000-fold permutation test. (B) Similarity values differed between models and controls across all highly significant (p < 0.001, permutation test) cortical electrodes (Friedman test: p = 0.018). Post-hoc, the bottom-up model was significantly less similar to sEEG than the audio envelope ($\rho_{bottom-up}$ median (Q1, Q3) = 0.026 (0.015, 0.038); $\rho_{control}$ = 0.026 (0.012, 0.048); n = 364; two-sided Wilcoxon signed-rank test: $p_{adj}$ < $10^{-5}$). (C) Top-down bias model similarity to neural signals. (D) Post-hoc, the top-down model was significantly more similar to sEEG than the audio envelope ($\rho_{top-down}$ = 0.028 (0.014, 0.050); $\rho_{control}$ = 0.018 (0.008, 0.034); n = 350; two-sided Wilcoxon signed-rank test: $p_{adj}$ < $10^{-5}$). (E) Dynamical systems SAMy (y-unit) model similarity to neural signals. (F) Post-hoc, SAMy had significantly higher similarity values than the audio envelope control condition ($\rho_{SAMy}$ = 0.034 (0.017, 0.061); $\rho_{control}$ = 0.022 (0.010, 0.041); n = 449; two-sided Wilcoxon signed-rank test: $p_{adj}$ < $10^{-5}$). SAMy had higher similarity values than both the top-down model ($\rho_{SAMy}$ = 0.034 (0.017, 0.061), $n_{SAMy}$ = 449; $\rho_{bottom-up}$ = 0.028 (0.014, 0.050), $n_{top-down}$ = 350; two-sided Wilcoxon signed-rank test: $p_{adj}$ = 0.003) and the bottom-up model ($\rho_{SAMy}$ = 0.034 (0.017, 0.061), $n_{SAMy}$ = 449; $\rho_{bottom-up}$ = 0.026 (0.015, 0.038), $n_{bottom-up}$ = 364; two-sided Wilcoxon signed rank test: $p_{adj}$ < $10^{-5}$). (G) Similarity values of highly significant (p < 0.001) electrodes by cortical region. Absolute value of the median $\rho$ coefficient is plotted by cortical region, based on a coarse regional atlas. Horizontal black bars represent region-wise medians. Statistics for within-region model comparisons are available in Table 1. See Supp. Fig. 1 for paired plots of model and control similarity values at the same electrode. All p-values are adjusted for multiple comparisons with a Holm-Bonferroni correction. Stars indicate significant differences between models within a region (∗ p < 0.05, ∗∗ p < 0.01, ∗∗∗ p < 0.001). All participants had sEEG coverage in the prefrontal and superior temporal cortex, but sEEG from other regions represents a subset of participants.


### imports

In [5]:
import numpy as np
import pandas as pd

import mne
from scipy.stats import zscore, false_discovery_control

from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns

### plotting functions

In [9]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def classify_pval(val):
    """classify p-value for sensor size"""
    if val > 0.05:
        return 0.5
    elif 0.001 < val <= 0.05:
        return 0.85
    elif val <= 0.001:
        return 1.25

def load_fif_epo(subj):
    """load iEEG epo.fif file
    
    Parameters:
    -----------
    subj : str
        subject ID number        
    Returns:
    --------
    epo : object
        instance of MNE Epochs 
    """
    
    path = '../../00-data/simulated_sEEG/sim/'+subj+'_sim_epo.fif' #'/Users/sydneysmith/Projects/PrOPHEcy/PrOPHEcy/00-data/epo_fifs/' + subj + '_BIP_epo.fif'
    epo = mne.read_epochs(path, preload=True)
    
    return epo

def plot_brain_model_fits(patients, SDids, metric, hemi, view):
    """Plots electrode locations from all iEEG recording onto single template brain
    NO NORMALIZATION per subject
    
    Paramters:
    ----------
    patients : list
        list of patient IDs to be plotted
    SDids : list
        list of strings of patient IDs from UCSD Health
    metric : str
        metric to plot or None
    hemi : str
        hemisphere to plot, "left" or "right"
    view : str
        view to plot, "medial" or "lateral"
    
        
    """
    subjects_dir = '../../00-data/freesurfer'#'/Users/sydneysmith/Projects/PrOPHEcy/3dSlicer_localization'
    
    # make brain surface
    brain = mne.viz.Brain(
            'fsaverage',
            title = 'Median '+metric+', n='+str(len(SDids)),
            hemi = 'both',
            surf='pial',
            subjects_dir=subjects_dir,
            cortex="low_contrast",
            alpha=0.2,
            background="white",
            units = 'm'
        )

    # load median model fits & pvals
    pval_dfs = []
    for patient in patients:
        pval_path = '../../03-results/perm_test/p_vals/'+patient+'_pvals.csv' #'/Users/sydneysmith/Projects/PrOPHEcy/PrOPHEcy/03-results/ieeg/perm_test/n_1000/p_vals/'+patient+'_pvals.csv'
        pval_df = pd.read_csv(pval_path)
        pval_df['patient'] = np.tile(patient, len(pval_df))
        pval_dfs.append(pval_df)

    all_pval_df = pd.concat(pval_dfs)

    # zscore metric across all patients
    all_pval_df[metric+'_zscore'] = zscore(abs(all_pval_df[metric].values))
    
    # loop through patients and add sensors (color code for metric value)
    for patient, SDid in zip(patients, SDids):

        # load epochs object for one patient
        epochs = load_fif_epo(patient)
        epochs = epochs.pick_types(seeg=True)
        epochs = epochs.drop_channels(epochs.info['bads'])

        # load similarity metrics for one patient
        model_path = '../../03-results/model_comparison/subj_'+patient+'_model_comparison.csv' #'/Users/sydneysmith/Projects/PrOPHEcy/PrOPHEcy/03-results/ieeg/model_comparison/subj_'+patient+'_model_comparison.csv'
        df = pd.read_csv(model_path) 

        subject = 'FSURF_'+SDid

        # channel locations
        montage = epochs.get_montage()
        ch_names = montage.ch_names
        ch_pos = montage.get_positions()['ch_pos'].values()
        ch_pos = list(ch_pos)
        ch_pos_new = [pos/1000 for pos in ch_pos] # adjust to meter scaling from mm

        # identify hemisphere from channel name
        ch_names_hemi = ['right' if (name[0]=='R') or (name[0]=='D') else 'left' for name in ch_names]
        ch_names_right = np.array(ch_names)[np.array([hemi=='right' for hemi in ch_names_hemi])]
        ch_names_left = np.array(ch_names)[np.array([hemi=='left' for hemi in ch_names_hemi])]
        
        if metric == 'None':
            vals = None
        else:
            # create df with median values across all channels
            median_df = pd.DataFrame(df.groupby('channel', sort=False, as_index=False)[metric].median())
            
            # get this patient's chunk of model dataframe
            subj_pval_df = all_pval_df[all_pval_df['patient']==patient][['channel', metric+'_pval', metric+'_zscore']]

            # FDR correction
            subj_pval_df['FDR_pval'] = false_discovery_control(subj_pval_df[metric+'_pval'].values, method='bh')

            # merge & compute sensor size factors
            median_df = pd.merge(median_df, subj_pval_df, how='left', on='channel')
            median_df['size_factor'] = median_df['FDR_pval'].apply(classify_pval)
            
            # select data from desired hemisphere
            if hemi=='right':
                median_df = median_df[median_df['channel'].isin(ch_names_right)]
                #vals = median_df[metric+'_normalized'].values
                vals = median_df[metric+'_zscore'].values
            elif hemi=='left':
                median_df = median_df[median_df['channel'].isin(ch_names_left)]
                #vals = median_df[metric+'_normalized'].values
                vals = median_df[metric+'_zscore'].values
                
        if len(vals)==0:
            continue
        else:
            if metric == 'raw_SAM_y_rho':
                colors = [cm.RdPu(i) for i in NormalizeData(vals)]
            elif metric == 'raw_top_rho':
                colors = [cm.OrRd(i) for i in NormalizeData(vals)]
            elif metric == 'raw_btm_rho':
                colors = [cm.Blues(i) for i in NormalizeData(vals)]
            elif metric == 'raw_SAM_i_rho':
                colors = [cm.Greens(i) for i in NormalizeData(vals)]
            elif metric == 'raw_SAM_e_rho':
                colors = [cm.YlOrBr(i) for i in NormalizeData(vals)]
            elif metric == 'None':
                colors = 'black'
            else:
                print('metric is '+metric)
                colors = [cm.viridis(i) for i in NormalizeData(vals)]
        
        features = ['raw_top_r', 'raw_top_rho',
            'raw_btm_r', 'raw_btm_rho', 
            'raw_comb_r', 'raw_comb_rho',
            'raw_SAM_y_r', 'raw_SAM_y_rho',
            'raw_SAM_e_r', 'raw_SAM_e_rho', 
            'raw_SAM_i_r', 'raw_SAM_i_rho']

    
        if patient == '1014':
            ch_pos_new = np.asarray(ch_pos_new)
            ch_pos_new[:,0] = ch_pos_new[:,0]+0.006
            ch_pos_new = ch_pos_new*0.92
            
            new_montage = mne.channels.make_dig_montage(dict(zip(ch_names, ch_pos_new)), coord_frame='mri')
            new_montage.add_estimated_fiducials(subject, subjects_dir)
            
            
        else:
        
            new_montage = mne.channels.make_dig_montage(dict(zip(ch_names, ch_pos_new)), coord_frame='mri')
            new_montage.add_estimated_fiducials(subject, subjects_dir)
    
            # Taliarach transform in FSURF_ folder
            mri_mni_t = mne.read_talxfm(subject, subjects_dir)
            new_montage.apply_trans(mri_mni_t)  # mri to mni_tal (MNI Taliarach)
    
            # transform to fsaverage
            new_montage.apply_trans(mne.transforms.Transform(fro="mni_tal", to="mri", trans=np.eye(4)))
        
        
        epochs.set_montage(new_montage)

        # exclude channels from other hemisphere
        if len(ch_names_right)==0:
            pass
        if len(ch_names_left)==0:
            continue
        else:
            if hemi == 'right':
                epochs.pick(list(ch_names_right))
            elif hemi=='left':
                epochs.pick(list(ch_names_left))
            else:
                print('hemi must be "left" or "right"') 
        
        trans = mne.channels.compute_native_head_t(new_montage)
                
        # plot projected channels on fsaverage
        brain.add_sensors(epochs.info, trans=trans, sensor_colors=colors, sensor_scales=0.003*median_df['size_factor'].values) #*median_df['size_factor'].values

    lh = list(brain.plotter.actors.keys())[0]
    rh = list(brain.plotter.actors.keys())[1]

    
    
    if (hemi == 'right') & (view == 'lateral'):
        brain.plotter.remove_actor(lh)
        brain.show_view(azimuth=0)
    elif (hemi == 'right') & (view == 'medial'):
        brain.plotter.remove_actor(lh)
        brain.show_view(azimuth=180, distance=0.38)
    elif (hemi == 'left') & (view == 'lateral'):
        brain.plotter.remove_actor(rh)
        brain.show_view(azimuth=180)
    elif (hemi == 'left') & (view == 'medial'):
        brain.plotter.remove_actor(rh)
        brain.show_view(azimuth=0, distance=0.38)
        
    # save image
    # brain.save_image()
        
     
    

## panel A

This panel contains sEEG electrode location information, and these data are available upon reasonable request, subject to IRB approval.

In [ ]:
patients = ['1002', '1005', '1007', '1008', '1009', '1010', '1014']
SDids = ['SD021', 'SD026', 'SD027', 'SD028', 'SD029', 'SD030', 'SD035']


metrics = ['raw_btm_rho']
hemis = ['left', 'right']
views = ['lateral']

for metric in metrics:
    for hemi in hemis:
        for view in views:
            plot_brain_model_fits(patients, SDids, metric=metric, hemi=hemi, view=view)

## panel B

## panel C

This panel contains sEEG electrode location information, and these data are available upon reasonable request, subject to IRB approval.

In [ ]:
patients = ['1002', '1005', '1007', '1008', '1009', '1010', '1014']
SDids = ['SD021', 'SD026', 'SD027', 'SD028', 'SD029', 'SD030', 'SD035']


metrics = ['raw_top_rho']
hemis = ['left', 'right']
views = ['lateral']

for metric in metrics:
    for hemi in hemis:
        for view in views:
            plot_brain_model_fits(patients, SDids, metric=metric, hemi=hemi, view=view)

## panel D

## panel E

This panel contains sEEG electrode location information, and these data are available upon reasonable request, subject to IRB approval.

In [ ]:
patients = ['1002', '1005', '1007', '1008', '1009', '1010', '1014']
SDids = ['SD021', 'SD026', 'SD027', 'SD028', 'SD029', 'SD030', 'SD035']


metrics = ['raw_SAM_y_rho',]
hemis = ['left', 'right']
views = ['lateral']

for metric in metrics:
    for hemi in hemis:
        for view in views:
            plot_brain_model_fits(patients, SDids, metric=metric, hemi=hemi, view=view)

## panel F


## panel G